 1. Sales Performance Analysis
Key Metrics:
- Sales by product (in gallons and units)
- Moving average volume by SKU (trend analysis)
- Average sales price per product/SKU
- Number of times each item was ordered (demand frequency)

Purpose:
- Identify top-selling products, pricing trends, and demand patterns.

Key Questions for Churn & Product Movement

How much of each product sold this quarter vs. last quarter?

Which products’ sales are growing, and which are declining?

How quickly are we turning inventory over for each product?

Which products have flat or declining movement and might be at risk of churn (slow movers)?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [2]:
directory = "/Users/nirugidla/PycharmProjects/UMICHIGAN_code/dashboards/"

duet_invoice_data_file = "DUET__Invoice_Line__c-7_24_2025.csv"

duet_invoice_df = pd.read_csv(os.path.join(directory, duet_invoice_data_file))

print(len(duet_invoice_df.columns), "duet_invoice columns")

19 duet_invoice columns


In [3]:
print(duet_invoice_df.shape, "duet invoice")

(68404, 19) duet invoice


In [4]:
duet_invoice_df.columns

Index(['_', 'Id', 'Name', 'CC_Order__c', 'CC_Order_Item__c', 'CC_Product__c',
       'Product_Code__c', 'Product_Description__c', 'Product_Family__c',
       'Product_Master_Code__c', 'Product_Name__c', 'Product_SKU__c',
       'Sales_Tax__c', 'Ship_To__c', 'Total_Cost__c', 'Total_Price__c',
       'CreatedDate', 'Bulk__c', 'BurUnitCost__c'],
      dtype='object')

In [5]:
def process_created_date(df):
    # Convert the CreatedDate column to datetime
    df['CreatedDate_dt'] = pd.to_datetime(df['CreatedDate'], errors='coerce')
    # New column with yyyy-mm-dd
    df['CreatedDate_ymd'] = df['CreatedDate_dt'].dt.strftime('%Y-%m-%d')
    # New column with year
    df['CreatedDate_year'] = df['CreatedDate_dt'].dt.year
    # New column with 3-letter month abbreviation
    df['CreatedDate_month'] = df['CreatedDate_dt'].dt.strftime('%b')
    return df
duet_invoice_df = process_created_date(duet_invoice_df)

In [6]:
duet_invoice_df[['CreatedDate', 'BurUnitCost__c', 'Total_Price__c', 'Total_Cost__c','Product_Description__c']]

,CreatedDate,BurUnitCost__c,Total_Price__c,Total_Cost__c,Product_Description__c
0,2023-05-24T15:18:13.000+0000,NaN,54180.00,NaN,Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF...
1,2023-05-24T15:18:13.000+0000,NaN,246.00,NaN,"Vitalube Cleaning Fluid, 6/1 Gal"
2,2023-05-24T15:18:13.000+0000,NaN,155.55,NaN,"Vitalube Elevator Cable Lubricant, 6/1 Gal"
3,2023-05-24T15:18:13.000+0000,NaN,123.00,NaN,"Vitalube Cleaning Fluid, 6/1 Gal"
4,2023-05-24T15:18:13.000+0000,NaN,615.00,NaN,"Vitalube Cleaning Fluid, 6/1 Gal"
...,...,...,...,...,...
68399,2025-06-27T21:01:11.000+0000,0.00000,0.00,0.00,-
68400,2025-06-27T21:01:14.000+0000,0.00000,0.00,0.00,"Top 1 Evolution HDMO Full Syn 5W-40 CK-4, 4/5 ..."
68401,2025-06-27T21:01:14.000+0000,0.00000,0.00,0.00,Top 1 Action Plus Syn-Blend MCO 2T JASO FD ESL...
68402,2025-06-27T21:01:14.000+0000,0.00000,0.00,0.00,"Top 1 VITA LL Gas Engine Oil SAE 40W, 55 Gal Drum"


In [7]:
duet_invoice_df[['CreatedDate', 'BurUnitCost__c', 'Total_Price__c', 'Total_Cost__c','Product_Description__c']].describe()

,BurUnitCost__c,Total_Price__c,Total_Cost__c
count,37295.000000,5.114200e+04,3.729600e+04
mean,1.584106,8.522056e+03,6.962957e+03
std,7.728378,1.799913e+04,2.040563e+04
min,0.000000,-5.919678e+04,0.000000e+00
25%,0.000000,6.677250e+02,5.910000e+02
50%,0.009920,3.745500e+03,2.874015e+03
75%,0.241905,1.125686e+04,8.572397e+03
max,495.340000,2.636600e+06,2.331420e+06


In [8]:
duet_invoice_df.head()

,_,Id,Name,CC_Order__c,CC_Order_Item__c,CC_Product__c,Product_Code__c,Product_Description__c,Product_Family__c,Product_Master_Code__c,...,Ship_To__c,Total_Cost__c,Total_Price__c,CreatedDate,Bulk__c,BurUnitCost__c,CreatedDate_dt,CreatedDate_ymd,CreatedDate_year,CreatedDate_month
0,[DUET__Invoice_Line__c],a4r5d000002ZKZCAA4,268332_1,a0n5d00000aAjWMAA0,NaN,a1F5d00000nfcP6EAI,NaN,Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF...,Motor Oil,ILA-LZ0W20-M,...,a025d000013B8J6AAK,NaN,54180.00,2023-05-24T15:18:13.000+0000,True,NaN,2023-05-24 15:18:13+00:00,2023-05-24,2023,May
1,[DUET__Invoice_Line__c],a4r5d000002ZKZDAA4,268334_1,a0n5d00000aAeL7AAK,NaN,a1F5d00000qScszEAC,-,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,60905DH,...,a025d000013AdPkAAK,NaN,246.00,2023-05-24T15:18:13.000+0000,False,NaN,2023-05-24 15:18:13+00:00,2023-05-24,2023,May
2,[DUET__Invoice_Line__c],a4r5d000002ZKZEAA4,268335_1,a0n5d00000aAegjAAC,NaN,a1F5d00000qScsxEAC,-,"Vitalube Elevator Cable Lubricant, 6/1 Gal",Industrial Oil,50020EN,...,a025d000013BhbWAAS,NaN,155.55,2023-05-24T15:18:13.000+0000,False,NaN,2023-05-24 15:18:13+00:00,2023-05-24,2023,May
3,[DUET__Invoice_Line__c],a4r5d000002ZKZFAA4,268336_1,a0n5d00000aAeygAAC,NaN,a1F5d00000qScszEAC,-,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,60905DH,...,a025d000013BjSLAA0,NaN,123.00,2023-05-24T15:18:13.000+0000,False,NaN,2023-05-24 15:18:13+00:00,2023-05-24,2023,May
4,[DUET__Invoice_Line__c],a4r5d000002ZKZGAA4,268337_1,a0n5d00000aAfphAAC,NaN,a1F5d00000qScszEAC,-,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,60905DH,...,a025d000013AeacAAC,NaN,615.00,2023-05-24T15:18:13.000+0000,False,NaN,2023-05-24 15:18:13+00:00,2023-05-24,2023,May


In [9]:
duet_invoice_df['CC_Product__c'].value_counts()

CC_Product__c
a1F3h0000003wM0EAI    1233
a1F5d00000nfcP6EAI     992
a1F3h0000003wpJEAQ     924
a1F3h0000003rElEAI     896
a1F3h0000003wpYEAQ     869
                      ... 
a1F5d00000nlqViEAI       1
a1F5d00000nket0EAA       1
a1FQk00000094fXMAQ       1
a1FQk00000094fWMAQ       1
a1F5d00000nffyoEAA       1
Name: count, Length: 2293, dtype: int64

In [10]:
duet_invoice_df['Product_Name__c'].value_counts()

Product_Name__c
-                 6873
3143-000          1233
ILA-LZ0W20         992
721047             924
7483-000           896
                  ... 
85911-005            1
DYN10000004977       1
5146000              1
5145900              1
DR1-00003            1
Name: count, Length: 2294, dtype: int64

In [11]:
# keep only rows where Product_Name__c is NOT exactly '-'
duet_invoice_df = duet_invoice_df[duet_invoice_df['Product_Name__c'] != '-']

In [12]:
['Sales_Tax__c', 'Ship_To__c', 'Total_Cost__c', 'Total_Price__c', 'BurUnitCost__c',
       'CreatedDate_ymd', 'CreatedDate_year', 'CreatedDate_month'] #numeric values or quantitative dat

['Sales_Tax__c',
 'Ship_To__c',
 'Total_Cost__c',
 'Total_Price__c',
 'BurUnitCost__c',
 'CreatedDate_ymd',
 'CreatedDate_year',
 'CreatedDate_month']

In [13]:
['Product_Description__c', 'Product_Family__c'] #categorical values or qualitative data

['Product_Description__c', 'Product_Family__c']

In [14]:
duet_invoice_df['Product'] = duet_invoice_df['Product_Description__c'].apply(lambda x: x.split(',')[0])
duet_invoice_df['Packaging'] = duet_invoice_df['Product_Description__c'].apply(lambda x: x.split(',')[-1])

In [15]:
num_cols = duet_invoice_df.select_dtypes(include='number').columns.tolist()
time_cols = ['CreatedDate_ymd', 'CreatedDate_month', 'CreatedDate_year']
bad_cols = ['_', 'Id', 'Name', 'CC_Order__c','CC_Order_Item__c', 'CC_Order_Item__c']
cat_cols = ['Product_Description__c', 'Product_Family__c', 'Packaging','Product']
numeric_cols = [n for n in num_cols if n not in time_cols and n not in bad_cols]

In [16]:
cat_cols

['Product_Description__c', 'Product_Family__c', 'Packaging', 'Product']

In [17]:
numeric_cols

['Sales_Tax__c', 'Total_Cost__c', 'Total_Price__c', 'BurUnitCost__c']

In [18]:
duet_invoice_df[numeric_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Sales_Tax__c,1659.0,105.576667,133.705300,2.31,35.81,75.04000,133.28500,3543.60
Total_Cost__c,36286.0,7023.257001,16229.960439,0.00,681.67,3185.27500,8738.51000,1171264.94
Total_Price__c,47740.0,9000.198765,13538.593156,-59196.78,1032.93,4617.45000,11850.60000,721712.16
BurUnitCost__c,36285.0,1.621578,7.820673,0.00,0.00,0.01077,0.30096,495.34


In [19]:
duet_invoice_df[numeric_cols + cat_cols].isna().sum()

Sales_Tax__c              59872
Total_Cost__c             25245
Total_Price__c            13791
BurUnitCost__c            25246
Product_Description__c        0
Product_Family__c             0
Packaging                     0
Product                       0
dtype: int64

In [20]:
for c in cat_cols:
    print(c, duet_invoice_df[c].nunique())
    display(duet_invoice_df[c].value_counts().head(10))

Product_Description__c 2281


Product_Description__c
Sunoco Super C Gold 15W-40 CK-4/SN , Bulk                  1233
Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF-6A, Bulk     992
Sunoco Ultra FS 5W-30 Dexos1/SP/GF-6A , 12/1 Qt             924
Sunoco Ultra SB SAE 5W-30 SQ/GF-7A , Bulk                   896
Sunoco Ultra SB SAE 5W-30 SP/GF-6A , 12/1 Qt                869
Sunoco Ultra FS 0W-20 Dexos1/SQ/GF-7A , 12/1 Qt             865
Sunoco Ultra SB SAE 5W-20 SQ/GF-7A, 12/1 Qt                 779
Sunoco Ultra SB SAE 5W-20 SQ/GF-7A, Bulk                    646
Sunoco Ultra FS 0W-20 Dexos1/SP/GF-6A , Bulk                632
Sunoco Sunvis 846, Bulk                                     623
Name: count, dtype: int64

Product_Family__c 11


Product_Family__c
Motor Oil             35086
Hydraulic Fluid        6635
Transmission Fluid     6432
Gear Oil               4269
Industrial Oil         4236
Grease                 2003
Process Oil            1912
Anti-Freeze             680
Additives               140
Equipment               135
Name: count, dtype: int64

Packaging 123


Packaging
Bulk           26347
12/1 Qt        11297
55 Gal Drum     9658
5 Gal Pail      3371
Epack           2349
275 Tote        1702
6/1 Gal          960
3/1 Gal          772
120 Lb Keg       711
14oz Pack        537
Name: count, dtype: int64

Product 1509


Product
Sunoco Super C Gold 15W-40 CK-4/SN                   1637
Sunoco Ultra SB SAE 5W-20 SQ/GF-7A                   1503
Sunoco Ultra FS 5W-30 Dexos1/SP/GF-6A                1404
Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF-6A     992
Sunoco Sunvis 846                                     972
Sunoco Ultra SB SAE 5W-30 SP/GF-6A                    920
Sunoco Ultra SB SAE 5W-30 SQ/GF-7A                    896
Sunoco Ultra FS 0W-20 Dexos1/SQ/GF-7A                 865
Sunoco Tacky Red Grease EP #2                         720
Sunoco Ultra FS 0W-20 Dexos1/SP/GF-6A                 717
Name: count, dtype: int64

In [21]:
duet_invoice_df[['Product_Description__c', 'Product_Family__c']]

,Product_Description__c,Product_Family__c
0,Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF...,Motor Oil
1,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil
2,"Vitalube Elevator Cable Lubricant, 6/1 Gal",Industrial Oil
3,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil
4,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil
...,...,...
68398,"Top 1 HP Plus Syn Blend 5W-30 SP - ESL, 12/1 L...",Motor Oil
68400,"Top 1 Evolution HDMO Full Syn 5W-40 CK-4, 4/5 ...",Motor Oil
68401,Top 1 Action Plus Syn-Blend MCO 2T JASO FD ESL...,Motor Oil
68402,"Top 1 VITA LL Gas Engine Oil SAE 40W, 55 Gal Drum",Motor Oil


In [22]:
duet_invoice_df['Product_Description__c'].value_counts()

Product_Description__c
Sunoco Super C Gold 15W-40 CK-4/SN , Bulk                      1233
Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF-6A, Bulk         992
Sunoco Ultra FS 5W-30 Dexos1/SP/GF-6A , 12/1 Qt                 924
Sunoco Ultra SB SAE 5W-30 SQ/GF-7A , Bulk                       896
Sunoco Ultra SB SAE 5W-30 SP/GF-6A , 12/1 Qt                    869
                                                               ... 
Sunoco R&O 320, 5 Gal Pail                                        1
Top 1 Action Plus Syn-Blend MCO 2T JASO FD, 12/1 Liter Case       1
UNITED Universal TH Fluid , 5 Gal Pail                            1
Sunoco Sunep 100, 275 Tote                                        1
New infinium Blue Drum                                            1
Name: count, Length: 2281, dtype: int64

In [23]:
duet_invoice_df['Product_Family__c'].value_counts()

Product_Family__c
Motor Oil             35086
Hydraulic Fluid        6635
Transmission Fluid     6432
Gear Oil               4269
Industrial Oil         4236
Grease                 2003
Process Oil            1912
Anti-Freeze             680
Additives               140
Equipment               135
Chemicals                 3
Name: count, dtype: int64

In [24]:
duet_invoice_df['Total_Cost__c'].value_counts()

Total_Cost__c
0.00        907
83.34       268
85.80       157
166.68      112
53.70        88
           ... 
5352.11       1
1579.49       1
2948.39       1
1649.73       1
25561.95      1
Name: count, Length: 27485, dtype: int64

In [25]:
duet_invoice_df['Total_Price__c'].value_counts()

Total_Price__c
0.00        1477
155.55       229
123.00       175
449.64       101
596.84        99
            ... 
14313.10       1
17578.08       1
1372.28        1
2154.96        1
28249.00       1
Name: count, Length: 25389, dtype: int64

In [26]:
duet_invoice_df[['Product_Description__c',
                 'Product_Family__c','Sales_Tax__c', 'Total_Cost__c',
                 'Total_Price__c', 'BurUnitCost__c']]

,Product_Description__c,Product_Family__c,Sales_Tax__c,Total_Cost__c,Total_Price__c,BurUnitCost__c
0,Idemitsu Full Synthetic Engine Oil 0W-20 SP/GF...,Motor Oil,NaN,NaN,54180.00,NaN
1,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,NaN,NaN,246.00,NaN
2,"Vitalube Elevator Cable Lubricant, 6/1 Gal",Industrial Oil,NaN,NaN,155.55,NaN
3,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,NaN,NaN,123.00,NaN
4,"Vitalube Cleaning Fluid, 6/1 Gal",Industrial Oil,NaN,NaN,615.00,NaN
...,...,...,...,...,...,...
68398,"Top 1 HP Plus Syn Blend 5W-30 SP - ESL, 12/1 L...",Motor Oil,NaN,0.00,0.00,0.00000
68400,"Top 1 Evolution HDMO Full Syn 5W-40 CK-4, 4/5 ...",Motor Oil,NaN,0.00,0.00,0.00000
68401,Top 1 Action Plus Syn-Blend MCO 2T JASO FD ESL...,Motor Oil,NaN,0.00,0.00,0.00000
68402,"Top 1 VITA LL Gas Engine Oil SAE 40W, 55 Gal Drum",Motor Oil,NaN,0.00,0.00,0.00000


In [27]:
duet_invoice_df = duet_invoice_df[['Sales_Tax__c', 'Total_Cost__c', 'Total_Price__c', 'BurUnitCost__c', 'Product_Family__c', 'Packaging', 'Product','CreatedDate_month', 'CreatedDate_year']]

In [29]:
duet_invoice_df.shape

(61531, 9)

In [30]:
#duet_invoice_df.to_csv(directory+"duet_invoice_cleaned.csv")